# MRes Project

### Without optimization:

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import matplotlib

from matplotlib.animation import FFMpegWriter

matplotlib.use('TkAgg')

#==========================================================
#                1. System parameters
#==========================================================
wc = 1.0 * 2 * np.pi  # cavity frequency
wa = 1.0 * 2 * np.pi  # atom frequency

g = 0.05 * 2 * np.pi       # coupling strength
zeta = 0.1 * 2 * np.pi     # coupling strength of ion-laser field
epsilon = 0.1 * 2 * np.pi  # cavity-laser field coupling strength
w0 = 1.0 * 2 * np.pi       # laser frequency

# Hilbert space dimension for the cavity
n_cavity = 75

#==========================================================
#              2. Define Operators & Hamiltonian
#==========================================================
sigma_minus = tensor(qeye(n_cavity), sigmam())  # atomic lowering operator
sigma_plus  = tensor(qeye(n_cavity), sigmap())  # atomic raising operator
sigma_z     = tensor(qeye(n_cavity), sigmaz())  # atomic z operator
a           = tensor(destroy(n_cavity), qeye(2))  # cavity annihilation operator

# Times for which the state should be evaluated
times = np.linspace(0, 100, 100)  # time array
tau   = wc * times

# Time-dependent coefficients as strings for QuTiP
H_drive = [
    [sigma_minus, f"{zeta/wc} * exp(1j * {w0/wc} * t)"],
    [sigma_plus,  f"{zeta/wc} * exp(-1j * {w0/wc} * t)"],
    [a.dag(),     f"{epsilon/wc} * exp(-1j * {w0/wc} * t)"],
    [a,           f"{epsilon/wc} * exp(1j * {w0/wc} * t)"]
]

# Hamiltonian (Jaynes-Cummings + driving, under RWA)
H_theoretical = [
    a.dag() * a
    + 0.5 * (wa/wc) * sigma_z
    + (g/wc) * (a.dag() * sigma_minus + a * sigma_plus)
] + H_drive

#==========================================================
#              3. Initial State & Evolution
#==========================================================
# Atom in ground state, cavity in vacuum (Fock |0>)
psi0 = tensor(basis(n_cavity, 0), basis(2, 0))

# Solve for expectations of interest
result = mesolve(H_theoretical, psi0, tau, c_ops=[], e_ops=[a.dag() * a, sigma_plus * sigma_minus])
n_c = result.expect[0]
n_a = result.expect[1]

# Also collect full states for Wigner function
output = mesolve(H_theoretical, psi0, tau, c_ops=[], e_ops=[])

#==========================================================
#        4. Compute Wigner Negativity for Final State
#==========================================================
final_state         = output.states[-1]       # final time state
rho_cavity_final    = ptrace(final_state, 0)  # reduce to cavity
xvec                = np.linspace(-3, 3, 200)
W                   = wigner(rho_cavity_final, xvec, xvec)
dx                  = xvec[1] - xvec[0]
negative_part       = W[W < 0]
negativity          = np.sum(np.abs(negative_part)) * (dx**2)

#==========================================================
#        5. Print results (same structure as BO code)
#==========================================================
print("=========================================================")
print(f"Best negative of Wigner negativity: {-negativity}")
print("Parameters:")
print(f"    g       = {g}")
print(f"    zeta    = {zeta}")
print(f"    epsilon = {epsilon}")
print(f"    w0      = {w0}")
print("=========================================================")

#==========================================================
#            6. Plotting results
#==========================================================
fig, axes = plt.subplots(1, 1, figsize=(10, 6))
axes.plot(times, n_c, label="Cavity photon number")
axes.plot(times, n_a, label="Atom excitation number")
axes.grid(True)
axes.legend(loc='best')
axes.set_xlabel("Time (arbitrary units)")
axes.set_ylabel("Occupation number")
axes.set_title("Occupation number with driving field")
plt.savefig('driven_JC_model.png')
plt.show()

#==========================================================
#     7. Animation of the Wigner function
#==========================================================
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=25.0)
xvec = np.linspace(-3, 3, 200)

writer = FFMpegWriter(fps=15, metadata=dict(artist='Me'), bitrate=1800)

rho_cavity = [ptrace(rho, 0) for rho in output.states]
fig, ani = anim_wigner(rho_cavity, xvec, xvec, projection='3d', colorbar=True, fig=fig, ax=ax)
ani.save('wigner_function_animation_driven.mp4', writer='ffmpeg', fps=15)

plt.show()

Best negative of Wigner negativity: -0.370295719012405
Parameters:
    g       = 0.3141592653589793
    zeta    = 0.6283185307179586
    epsilon = 0.6283185307179586
    w0      = 6.283185307179586


### Implementing Bayesian Optimization:

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
import matplotlib

from matplotlib.animation import FFMpegWriter

matplotlib.use('TkAgg')

#==========================================================
#           1. Define the *original* system setup
#==========================================================

# Frequencies, couplings, etc. (in units of some frequency scale, e.g. 2*pi MHz)
wc_default = 1.0 * 2 * np.pi  # cavity frequency
wa_default = 1.0 * 2 * np.pi  # atom frequency

# These below will be replaced/optimized:
g_default       = 0.05 * 2 * np.pi  # coupling strength between atom and cavity
zeta_default    = 0.1  * 2 * np.pi  # ion-laser field coupling
epsilon_default = 0.1  * 2 * np.pi  # cavity-laser field coupling
w0_default      = 1.0  * 2 * np.pi  # laser frequency

# Hilbert space dimension for the cavity
n_cavity = 75

# Times for which the state should be evaluated
times = np.linspace(0, 100, 100)  # time array
# We'll define tau as wc * times once we fix wc.

# Initial state: atom in the ground state, cavity in the |0> Fock state
psi0 = tensor(basis(n_cavity, 0), basis(2, 0))

# Increase ODE solver steps to avoid "Excess work done" error
# ----------------------------------------------------------
solver_options = {"nsteps": 50000} # might increase

#==========================================================
#      2. Utility function: run the JCM + get negativity
#==========================================================
def run_JCM_and_get_negativity(params):
    """
    Evolve the Jaynes-Cummings system with driving for given parameters,
    and compute the Wigner negativity at the final time.

    Parameters
    ----------
    params : array-like of length 4
        [g, zeta, epsilon, w0], each in (some physically relevant range)

    Returns
    -------
    negativity : float
        The integral of the negative region of the Wigner function of the
        cavity state at the final time.
    """
    # Unpack parameters
    g_val, zeta_val, epsilon_val, w0_val = params

    # Frequencies
    wc = wc_default
    wa = wa_default

    # Operators
    a  = tensor(destroy(n_cavity), qeye(2))  # cavity annihilation
    sm = tensor(qeye(n_cavity), sigmam())    # atomic lowering
    sp = tensor(qeye(n_cavity), sigmap())    # atomic raising
    sz = tensor(qeye(n_cavity), sigmaz())    # atomic sigma_z

    # Time array for mesolve is scaled by wc
    tau = wc * times

    # Time-dependent drive (string format for QuTiP)
    H_drive = [
        [sm, f"{zeta_val/wc} * exp(1j * {w0_val/wc} * t)"],
        [sp, f"{zeta_val/wc} * exp(-1j * {w0_val/wc} * t)"],
        [a.dag(), f"{epsilon_val/wc} * exp(-1j * {w0_val/wc} * t)"],
        [a,      f"{epsilon_val/wc} * exp(1j * {w0_val/wc} * t)"]
    ]

    # RWA Jaynes-Cummings Hamiltonian + driving
    H_JC = [
        a.dag() * a
        + 0.5 * (wa / wc) * sz
        + (g_val / wc) * (a.dag() * sm + a * sp)
    ] + H_drive

    # Solve the Schrodinger equation (use large nsteps to avoid integrator exception)
    output = mesolve(H_JC, psi0, tau, c_ops=[], e_ops=[], options=solver_options)

    # Reduced final state
    final_state       = output.states[-1]
    rho_cavity_final  = ptrace(final_state, 0)

    # Compute Wigner negativity
    xvec = np.linspace(-3, 3, 200)
    W    = wigner(rho_cavity_final, xvec, xvec)
    dx   = xvec[1] - xvec[0]

    negative_part = W[W < 0]
    negativity    = np.sum(np.abs(negative_part)) * (dx ** 2)

    return negativity

#==========================================================
#     3. Bayesian Optimization: define objective & run
#==========================================================

# Parameter search ranges;
space  = [
    Real(0.01 * 2*np.pi, 0.2 * 2*np.pi,   name='g'),       # range for g
    Real(0.01 * 2*np.pi, 0.2 * 2*np.pi,   name='zeta'),    # range for zeta
    Real(0.01 * 2*np.pi, 0.2 * 2*np.pi,   name='epsilon'), # range for epsilon
    Real(0.8  * 2*np.pi, 1.2 * 2*np.pi,   name='w0'),      # range for w0
]

@use_named_args(space)
def objective(**kwargs):
    """
    Bayesian Optimization objective function:
    We want to *maximize* Wigner negativity, so we
    *return the negative* of that negativity to let
    the optimizer do minimization.
    """
    g_       = kwargs['g']
    zeta_    = kwargs['zeta']
    epsilon_ = kwargs['epsilon']
    w0_      = kwargs['w0']

    negativity = run_JCM_and_get_negativity([g_, zeta_, epsilon_, w0_])

    # Return negative of negativity for the minimizer
    return -negativity

# Perform gp_minimize
res = gp_minimize(objective, dimensions=space, n_calls=200, n_random_starts=10, random_state=0)

print("=========================================================")
print("Bayesian Optimization completed.")
print(f"Best negative of Wigner negativity found: {res.fun}")
print("Optimized parameters:")
print(f"    g       = {res.x[0]}")
print(f"    zeta    = {res.x[1]}")
print(f"    epsilon = {res.x[2]}")
print(f"    w0      = {res.x[3]}")
print("=========================================================")

#==========================================================
#         4. Re-run with best found parameters
#==========================================================
g_opt, zeta_opt, epsilon_opt, w0_opt = res.x

# Create operators with optimized parameters
a  = tensor(destroy(n_cavity), qeye(2))
sm = tensor(qeye(n_cavity), sigmam())
sp = tensor(qeye(n_cavity), sigmap())
sz = tensor(qeye(n_cavity), sigmaz())

wc  = wc_default
wa  = wa_default
tau = wc * times

H_drive_opt = [
    [sm, f"{zeta_opt/wc} * exp(1j * {w0_opt/wc} * t)"],
    [sp, f"{zeta_opt/wc} * exp(-1j * {w0_opt/wc} * t)"],
    [a.dag(), f"{epsilon_opt/wc} * exp(-1j * {w0_opt/wc} * t)"],
    [a,       f"{epsilon_opt/wc} * exp(1j * {w0_opt/wc} * t)"]
]

H_theoretical_opt = [
    a.dag() * a
    + 0.5 * (wa/wc) * sz
    + (g_opt / wc) * (a.dag() * sm + a * sp)
] + H_drive_opt

# Solve the Schrödinger equation with optimized parameters (with large nsteps)
result_opt = mesolve(H_theoretical_opt, psi0, tau, c_ops=[], e_ops=[a.dag() * a, sp * sm], options=solver_options)
output_opt = mesolve(H_theoretical_opt, psi0, tau, c_ops=[], e_ops=[], options=solver_options)

# Extract expectation values
n_c_opt = result_opt.expect[0]
n_a_opt = result_opt.expect[1]

#==========================================================
#              5. Plot the results
#==========================================================
fig, axes = plt.subplots(1, 1, figsize=(10, 6))
axes.plot(times, n_c_opt, label="Cavity photon number")
axes.plot(times, n_a_opt, label="Atom excitation number")
axes.grid(True)
axes.legend(loc='best')
axes.set_xlabel("Time (arbitrary units)")
axes.set_ylabel("Occupation number")
axes.set_title("Occupation number with driving field (Optimized)")
plt.savefig('driven_JC_model.png')
plt.show()

#==========================================================
#    6. Animation of the Wigner function with best params
#==========================================================
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=25.0)
xvec = np.linspace(-3, 3, 200)

writer = FFMpegWriter(fps=15, metadata=dict(artist='Me'), bitrate=1800)

rho_cavity_opt = [ptrace(rho, 0) for rho in output_opt.states]
fig, ani = anim_wigner(rho_cavity_opt, xvec, xvec, projection='3d', colorbar=True, fig=fig, ax=ax)
ani.save('wigner_function_animation_driven.mp4', writer='ffmpeg', fps=15)

plt.show()

Bayesian Optimization completed.
Best negative of Wigner negativity found: -0.5059885877622357
Optimized parameters:
    g       = 1.2566370614359172
    zeta    = 0.06283185307179587
    epsilon = 0.5025892887733358
    w0      = 6.273776246381983
